In [ ]:
import pandas as pd
import os
import tempfile
import requests
import json
import re
from urllib.request import urlretrieve
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from langchain_core.documents import Document
from tqdm.auto import tqdm

INFO 02-11 04:38:43 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
df = pd.read_json("real_final_chunks.jsonl", lines=True)
df

,drug_id,chunk_id,drug_name,drug_name_key,pro_type,active_ingredient,section,sub_index,sub_count,text
0,1901ac0340da,1901ac0340da-dose-1,"1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...","1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...",일반의약품,[M215884]포비돈 요오드,용법용량,1,1,"【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이..."
1,1901ac0340da,1901ac0340da-warn-1,"1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...","1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...",일반의약품,[M215884]포비돈 요오드,주의사항,1,4,"【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이..."
2,1901ac0340da,1901ac0340da-warn-2,"1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...","1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...",일반의약품,[M215884]포비돈 요오드,주의사항,2,4,"【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이..."
3,1901ac0340da,1901ac0340da-warn-3,"1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...","1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...",일반의약품,[M215884]포비돈 요오드,주의사항,3,4,"【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이..."
4,1901ac0340da,1901ac0340da-warn-4,"1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...","1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈...",일반의약품,[M215884]포비돈 요오드,주의사항,4,4,"【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이..."
...,...,...,...,...,...,...,...,...,...,...
64010,ceae1435068f,ceae1435068f-warn-2,힘엔스틱액(L-아스파르트산-L-아르기닌수화물),힘엔스틱액(L-아스파르트산-L-아르기닌수화물),일반의약품,[M258544]L-아스파르트산-L-아르기닌수화물,주의사항,2,6,【약물명】힘엔스틱액(L-아스파르트산-L-아르기닌수화물)\n【주성분】[M258544]...
64011,ceae1435068f,ceae1435068f-warn-3,힘엔스틱액(L-아스파르트산-L-아르기닌수화물),힘엔스틱액(L-아스파르트산-L-아르기닌수화물),일반의약품,[M258544]L-아스파르트산-L-아르기닌수화물,주의사항,3,6,【약물명】힘엔스틱액(L-아스파르트산-L-아르기닌수화물)\n【주성분】[M258544]...
64012,ceae1435068f,ceae1435068f-warn-4,힘엔스틱액(L-아스파르트산-L-아르기닌수화물),힘엔스틱액(L-아스파르트산-L-아르기닌수화물),일반의약품,[M258544]L-아스파르트산-L-아르기닌수화물,주의사항,4,6,【약물명】힘엔스틱액(L-아스파르트산-L-아르기닌수화물)\n【주성분】[M258544]...
64013,ceae1435068f,ceae1435068f-warn-5,힘엔스틱액(L-아스파르트산-L-아르기닌수화물),힘엔스틱액(L-아스파르트산-L-아르기닌수화물),일반의약품,[M258544]L-아스파르트산-L-아르기닌수화물,주의사항,5,6,【약물명】힘엔스틱액(L-아스파르트산-L-아르기닌수화물)\n【주성분】[M258544]...


In [ ]:
docs = []
with open("/workspace/real_final_chunks.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)

        docs.append(
            Document(
                page_content=obj["text"],
                metadata={ 
                    "drug_id": obj.get("drug_id"),
                    "chunk_id": obj.get("chunk_id"),
                    "drug_name": obj.get("drug_name"),
                    "drug_name_key": obj.get("drug_name_key"),
                    "pro_type": obj.get("pro_type"),
                    "active_ingredient": obj.get("active_ingredient"),
                    "section": obj.get("section"),
                    "sub_index": obj.get("sub_index"),
                    "sub_count": obj.get("sub_count"),
                }
            )
        )

print("docs 개수:", len(docs))
print("첫 doc metadata:", docs[0].metadata)
print("첫 doc text:", docs[0].page_content[:80])

drug_names = sorted({d.metadata.get("drug_name") for d in docs if d.metadata.get("drug_name")})
print("drug_names 개수:", len(drug_names))
print("예시:", drug_names[:10])

docs 개수: 64015
첫 doc metadata: {'drug_id': '1901ac0340da', 'chunk_id': '1901ac0340da-dose-1', 'drug_name': '1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)', 'drug_name_key': '1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)', 'pro_type': '일반의약품', 'active_ingredient': '[M215884]포비돈 요오드', 'section': '용법용량', 'sub_index': 1, 'sub_count': 1}
첫 doc text: 【약물명】1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)
【주성분】[M215884]포비돈 
drug_names 개수: 8649
예시: ['1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)', '1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀렌산나트륨오수화물)', '1.셀레뉴원오랄액(아셀렌산나트륨오수화물) 2.셀큐민185오랄액(아셀렌산나트륨오수화물)', '1.큐앤큐포비돈거즈볼-중(포비돈요오드)2.큐앤큐포비돈거즈볼-대(포비돈요오드)', '1.포리비돈인후스프레이(포비돈요오드)(바닐라향), 2.포리비돈인후스프레이(포비돈요오드)(청포도향)', '5-엠씨크림(리도카인)', '가그린목액', '가네리버연질캡슐175mg(밀크시슬열매건조엑스)', '가네리버연질캡슐350mg(밀크시슬열매건조엑스)', '가네맥스연질캡슐(밀크시슬열매건조엑스)']


- 메타데이터에서 약물명 추출

In [ ]:
drug_names = sorted({d.metadata["drug_name"] for d in docs if d.metadata.get("drug_name")})

- 벡터DB 생성

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

persist_dir = "/workspace/chroma_otc"
collection_name = "otc_chunks"


vectorstore = Chroma(
    persist_directory=persist_dir,
    collection_name=collection_name,
    embedding_function=embeddings,
)

BATCH = 256 
for i in tqdm(range(0, len(docs), BATCH), desc="Adding documents"):
    batch_docs = docs[i:i+BATCH]
    vectorstore.add_documents(batch_docs)

vectorstore.persist()
print("Saved to:", persist_dir)
print("Chroma count:", vectorstore._collection.count())

/tmp/ipykernel_4189/117949537.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/tmp/ipykernel_4189/117949537.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


Adding documents:   0%|          | 0/251 [00:00<?, ?it/s]

Saved to: /workspace/chroma_otc
Chroma count: 64015


/tmp/ipykernel_4189/117949537.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
def candidate_drugs(token: str, drug_names: list[str], top_n: int = 10):
    token = token.strip()
    cands = [d for d in drug_names if token in d]
    cands = sorted(cands, key=len)
    return cands[:top_n]

def choose_best_token(user_query: str, drug_names: list[str], top_n: int = 10):
    tokens = re.findall(r"[가-힣A-Za-z0-9]+", user_query)
    tokens = [t for t in tokens if len(t) >= 2]

    best_token, best_cands = None, []
    for t in tokens:
        cands = candidate_drugs(t, drug_names, top_n=top_n)
        if len(cands) > len(best_cands):
            best_token, best_cands = t, cands

    return best_token, best_cands

def pick_drug_name(user_query: str, top_n=10):
    best_token, cands, tokens = choose_best_token(user_query, top_n=top_n)

    print("\n[token 후보]:", tokens[:15])
    print("[선택된 token]:", best_token, "| 후보 수:", len(cands))

    if not cands:
        print("[drug_name 후보 없음] -> drug_name 필터 없이 진행")
        return None

    print("\n[drug_name 후보 목록]")
    for idx, name in enumerate(cands, 1):
        print(f"{idx}. {name}")

    while True:
        sel = input("번호 선택(Enter=1): ").strip()
        if sel == "":
            sel_idx = 1
            break
        if sel.isdigit():
            sel_idx = int(sel)
            if 1 <= sel_idx <= len(cands):
                break
        print(f"1~{len(cands)} 사이 숫자를 입력하세요.")

    chosen = cands[sel_idx - 1]
    print("선택된 drug_name:", chosen)
    return chosen

def search_from_user_query(user_query: str, k=6, top_n=10, interactive=True):
    chosen_drug_name = pick_drug_name(user_query, drug_names, top_n=top_n, interactive=interactive)

    if chosen_drug_name:
        hits = vectorstore.similarity_search(user_query, k=k, filter={"drug_name": chosen_drug_name})
        print("[필터검색] hits:", len(hits))
        return chosen_drug_name, hits

    hits = vectorstore.similarity_search(user_query, k=k)
    print("[전체검색] hits:", len(hits))
    return None, hits


In [ ]:
_, hits = search_from_user_query("음주 전후에 타이레놀 먹어도돼?", k=6, top_n=10, interactive=True)

[token 후보]: ['음주', '전후에', '타이레놀', '먹어도돼']
[선택된 token]: 타이레놀 | 후보 수: 7
1. 우먼스타이레놀정
2. 타이레놀콜드-에스정
3. 어린이타이레놀현탁액(아세트아미노펜)
4. 타이레놀8시간이알서방정(아세트아미노펜)
5. 타이레놀산500밀리그램(아세트아미노펜)
6. 타이레놀정500밀리그람(아세트아미노펜)
7. 어린이타이레놀산160밀리그램(아세트아미노펜)


번호 선택(Enter=1):  5


선택된 drug_name: 타이레놀산500밀리그램(아세트아미노펜)
[필터검색] hits: 6


In [ ]:
print(f"검색된 문서 수: {len(hits)}")
print(hits[0].metadata)
print(hits[0].page_content[:200])

검색된 문서 수: 6
{'sub_count': 6, 'pro_type': '일반의약품', 'chunk_id': 'a93187414074-warn-1', 'drug_id': 'a93187414074', 'drug_name': '타이레놀산500밀리그램(아세트아미노펜)', 'drug_name_key': '타이레놀산500밀리그램(아세트아미노펜)', 'section': '주의사항', 'sub_index': 1, 'active_ingredient': '[M262287]아세트아미노펜 과립'}
【약물명】타이레놀산500밀리그램(아세트아미노펜)
【주성분】[M262287]아세트아미노펜 과립
【구분】일반의약품
【섹션】주의사항 (1/6)
---
1. 경고
1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열 진통제를 복용해야 할 경우 반드시 의
사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 간손상이 유발될 수 있다.

